In [ ]:
!pip install category-encoders

In [ ]:
import pandas as pd
import numpy as np

import re
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler ,StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from category_encoders import HashingEncoder
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [ ]:
import csv
data = pd.read_csv("ApartmentRentPrediction.csv")
data["price_display"].isnull().sum()

FileNotFoundError: [Errno 2] No such file or directory: 'ApartmentRentPrediction.csv'

droping nulls from Y

In [ ]:
#nully = proj["price_display"].isnull()
#proj = proj[~nully]
data = data.dropna(subset=["price_display"])
proj = data.copy()

index 12 is the target

In [ ]:
x = proj.iloc[:, list(range(0, 11)) + list(range(13, 22))]
x['time'] = pd.to_datetime(x['time']).dt.time
y = proj["price_display"].str.replace(r'\D+','', regex=True).astype(int)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
def extract_beds(text):
    matches_numeric = re.findall(r'(\d+) beds?', text, re.IGNORECASE)  # Ignorecase for makig beds not case sensitive
    if matches_numeric:
        return int(matches_numeric[0])


    number_words = {'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10}
    for word, number in number_words.items():
        if word in text.lower():
            return number

    return None
x['beds'] = x['body'].apply(extract_beds)

In [ ]:
x.isnull().sum()

In [ ]:
y.isnull().sum()

In [ ]:
x_train.drop(columns=["time"], inplace=True)

In [ ]:
import datetime
def replace_nulls(df, type='test', X_trainApply=x_train):
  if type == 'train':
    numeric_columns = df.select_dtypes(include=['number']).columns
    string_columns = df.select_dtypes(exclude=['number']).columns
    for ncol in numeric_columns:
        df[ncol] = df[ncol].fillna(df[ncol].mean())
    for scol in string_columns:
      df[scol] = df[scol].fillna(df[scol].mode()[0])
  else :
    numeric_columns = df.select_dtypes(include=['number']).columns
    string_columns = df.select_dtypes(exclude=['number']).columns
    for ncol in numeric_columns:
        df[ncol] = df[ncol].fillna(X_trainApply[ncol].mean())
    for scol in string_columns:
      df[scol] = df[scol].fillna(X_trainApply[scol].mode()[0])

replace_nulls(x_train, 'train')

In [ ]:
def outlier_detection(df):
  if x is not None:
    numeric_columns = df.select_dtypes(include=['number']).columns
    for col in numeric_columns:
      Q1 = df[col].quantile(0.25)
      Q3 = df[col].quantile(0.75)
      IQR = Q3 - Q1
      outliers =(df[col] < (Q1 - 1.5 * IQR)) | (df[col] >( Q3 + 1.5 * IQR))
      print(f"{col}: {outliers.sum()}")

In [ ]:
outlier_detection(x)

In [ ]:
def replace_outliers(df):
  numeric_columns = df.select_dtypes(include=['number']).columns
  for col in numeric_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df[col] = np.where((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)), Q3 , df[col])

In [ ]:
#replace_outliers(x)

In [ ]:
x.nunique()

In [ ]:
columns = ['cityname','amenities' , 'address']
target_encoder = ce.TargetEncoder()
for col in columns:
  x_encoded = target_encoder.fit_transform(x[col], y)
  x[col] = x_encoded[col]

with open("target_encoder.pkl", "wb") as file:
    pickle.dump(target_encoder, file)

In [ ]:
x

In [ ]:
import pickle
from sklearn.preprocessing import LabelEncoder

cols=['pets_allowed', 'price_type','fee','currency','has_photo','amenities', 'address', 'body', 'category', 'cityname', 'state', 'source', 'title']
label_encoders = {}

for col in cols:
    label_encoder = LabelEncoder()
    x_train[col] = label_encoder.fit_transform(x_train[col])
    label_encoders[col] = label_encoder

with open('label_encoders.pkl', 'wb') as file:
    pickle.dump(label_encoders, file)

In [ ]:
x.head()

In [ ]:
x_train.info()

In [ ]:
x.head()

In [ ]:
x.nunique()

In [ ]:
x_train = x_train.drop(['id','currency' , 'fee'], axis=1)

In [ ]:
scaler = MinMaxScaler()
x_train[x_train.columns] = scaler.fit_transform(x_train[x_train.columns])
with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [ ]:
x_train

In [ ]:
def correlation(x, y):
  df = pd.concat([x, y], axis=1)
  corr = df.corr()
  top_feature = corr.index[abs(corr['price_display'])>0.2]

  plt.figure(figsize=(16, 14))
  sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".4f")
  plt.show()
  top_feature = top_feature.delete(-1)
  return x[top_feature]


In [ ]:
X_train = x_train
x_train = correlation(x_train, y_train)
with open("correlation_function.pkl", "wb") as file:
    pickle.dump(correlation, file)

In [ ]:
x_train.head()

In [ ]:
# Define the regression models
models = [
    ('Linear Regression', LinearRegression()),
    ('Ridge Regression', RidgeCV()),
    ('Lasso Regression', LassoCV()),
    ('Polynomial Regression', PolynomialFeatures(degree=2)),
    ('Random Forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('Decision Tree', DecisionTreeRegressor())
]


# Iterate over each model
for name, model in models:
    # Check if the current model is Polynomial Regression, Random Forest, or Decision Tree
    if name in ['Polynomial Regression', 'Random Forest', 'Decision Tree']:
        # For Polynomial Regression, create polynomial features and use Linear Regression
        if name == 'Polynomial Regression':
            poly = PolynomialFeatures(degree=2)
            x_poly = poly.fit_transform(x_train)
            model = LinearRegression()
        # For Random Forest, use RandomForestRegressor
        elif name == 'Random Forest':
            model = RandomForestRegressor(n_estimators=100, random_state=42)
        # For Decision Tree, use DecisionTreeRegressor
        else:
            model = DecisionTreeRegressor()

    #     # Perform Recursive Feature Elimination with Cross-Validation (RFECV)
    #     rfecv = RFECV(estimator=model, step=1, cv=5, scoring='neg_mean_squared_error')
    #     rfecv.fit(x_train, y_train)
    #     x_selected = x_train.iloc[:, rfecv.support_]
    # else:
    #     # For other models, perform RFECV without modifying the model
    #     rfecv = RFECV(estimator=model, step=1, cv=5, scoring='neg_mean_squared_error')
    #     rfecv.fit(x_train, y_train)
    #     x_selected = x_train.iloc[:, rfecv.support_]

    # Fit the model on the selected features
    model.fit(x_train, y_train)
    y_pred = model.predict(x_train)
    mse = ((y_train - y_pred) ** 2).mean()

    r2 = r2_score(y_train, y_pred)

    # Fit the model on the training data
    model.fit(x_train, y_train)
    with open(f"{name}_trained_model.pkl", "wb") as file:
      pickle.dump(model, file)
    y_pred_train = model.predict(x_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)

apply random forest because it's more accurate from linear regression because the data non-linear (the corelation with target not close to (1,-1))

and more effecient than decision tree because that work with multiple trees and that will reduce the overfitting

apply feature importance

In [ ]:
new_data = pd.read_csv("/content/ApartmentRentPrediction_test.csv")
new_data = new_data.dropna(subset=["price_display"])

model_names = ['Linear Regression', 'Ridge Regression', 'Lasso Regression',
               'Polynomial Regression', 'Random Forest', 'Decision Tree']

new_data_features = pd.DataFrame(new_data.loc[:, ~new_data.columns.isin(["price_display","price", "time","id","currency" , "fee"])])
new_data_target = new_data["price_display"]
replace_nulls(new_data_features)

with open("target_encoder.pkl", "rb") as file:
    loaded_target_encoder = pickle.load(file)

columns = ['cityname', 'amenities', 'Address']
missing_columns = [col for col in columns if col not in new_data.columns]
if not missing_columns:
    for col in columns:
        new_data[col] = loaded_target_encoder.transform(new_data[col])

labelencoders = pickle.load(open('label_encoders.pkl', 'rb'))

cols=['pets_allowed', 'price_type','has_photo','amenities', 'address', 'body', 'category', 'cityname', 'state', 'source', 'title']
for col in cols:
    new_data_features[col] = new_data_features[col].apply(lambda s: labelencoders[col].transform([s])[0] if s in labelencoders[col].classes_ else s)


for col in cols:
  max_value_train = X_train[col].max()
  unique_values_test = new_data_features[col].unique()
  for value in unique_values_test:
    if value not in X_train[col].unique():
      max_value_train += 1
      new_data_features.loc[new_data[col] == value, col] = max_value_train


In [ ]:
with open("scaler.pkl", "rb") as file:
    loaded_scaler = pickle.load(file)

columns_to_scale = X_train.columns
new_data_features[columns_to_scale] = loaded_scaler.transform(new_data_features[columns_to_scale])
new_data_target = pd.DataFrame(new_data_target)
new_data_target = new_data_target["price_display"].str.replace(r'\D+','', regex=True).astype(int)

with open("correlation_function.pkl", "rb") as file:
    loaded_correlation = pickle.load(file)
new_data_features = loaded_correlation(new_data_features,new_data_target)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
new_data_features = pd.DataFrame(new_data.loc[:, ~new_data.columns.isin(["RentCategory", "time"])])
for name in ['Linear Regression', 'Ridge Regression', 'Lasso Regression', 'Polynomial Regression', 'Random Forest', 'Decision Tree']:
    # Load the trained model from the pickle file
    model_filename = f"{name}_trained_model.pkl"
    with open(model_filename, "rb") as file:
        model = pickle.load(file)

    # Make predictions on the new data using the loaded model
    y_pred_new = model.predict(new_data_features)

    # Compute regression metrics
    mse = mean_squared_error(new_data_target, y_pred_new)
    mae = mean_absolute_error(new_data_target, y_pred_new)
    r2 = r2_score(new_data_target, y_pred_new)

    # Print the regression metrics
    print(f"Regression Metrics for {name}:")
    print("Mean Squared Error:", mse)
    print("Mean Absolute Error:", mae)
    print("R-squared Score:", r2)
    print()
